In [1]:
# requires Sentiment Analysis to have transformed interactions table

import pandas as pd
import sqlite3
import os
import matplotlib.pyplot as plt
import numpy as np
from dbl_constants import airlines, lang_to_tag, tag_to_lang, KLM
import re
from datetime import datetime
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.subplots as sp
from pandas.api.types import CategoricalDtype
import sqlite3
import dbl_constants
from typing import Iterable

database_path = "temporary_database.sqlite3"
used_languages = ["en"]


conn = sqlite3.connect(database_path)
cur = conn.cursor()
from IPython.core.display import display, HTML

def str_to_readable(s:str):
    return re.sub(r'([a-z](?=[A-Z])|[A-Z](?=[A-Z][a-z]))', r'\1 ', s.replace('_',' '))


def prepare_prompt(p):
    """Replaces certain strings in prompt"""

    def iter_to_sql_str(it: Iterable, s: str = '%s') -> str:
        """"Returns a string which can be used in sql WHERE IN ..."""
        st = ''
        li = list(it)
        for i in li[:-1]:
            st += s % str(i) + ", "
        st += s % str(li[-1])
        st = "(%s)" % st
        return st

    p = p.replace("[relevant_languages]", iter_to_sql_str(used_languages, "'%s'"))  # why "''" and not just ""
    p = p.replace("[relevant_users]", iter_to_sql_str(airlines.values(), "%s"))
    return p

database_path = "temporary_database.sqlite3"
conn = sqlite3.connect(database_path)
cur = conn.cursor()

cur.execute("""PRAGMA table_info(TWEETS);""")
tweet_columns = tuple(t[1] for t in cur.fetchall())
cur.execute("""PRAGMA table_info(USERS);""")
user_columns = tuple(t[1] for t in cur.fetchall())
cur.execute("""PRAGMA table_info(EXPANDED_INTERACTIONS);""")
expanded_interaction_columns = tuple(t[1] for t in cur.fetchall())


In [2]:
# cur.execute("""SELECT * FROM EXPANDED_INTERACTIONS""")
# interactions = pd.DataFrame(cur.fetchall(), columns=expanded_interaction_columns)

In [3]:
cur.execute(prepare_prompt("""SELECT * FROM TWEETS WHERE user_id in [relevant_users]"""))        
airline_tweets = pd.DataFrame(cur.fetchall(), columns=tweet_columns)
airline_tweets

,created_at,id,text,extended_fulltext,place_id,source,truncated,in_reply_to_status_id,in_reply_to_user_id,user_id,...,retweet_count,favorite_count,favorited,retweeted,filter_level,lang,timestamp_ms,quoted_status_id,hashtags,user_mentions
0,2019-05-22 12:21:58,1131173353291767810,@journoian We appreciate your feedback and we'...,None,None,"<a href=""http://www.conversocial.com"" rel=""nof...",0,1.131113e+18,4.083070e+07,18332190,...,0,0,0,0,low,en,1558527718266,NaN,,40830697
1,2019-05-22 12:22:25,1131173469339693057,@Shakal97883287 Oh no! Please can you drop us ...,None,None,"<a href=""http://www.conversocial.com"" rel=""nof...",0,1.130939e+18,1.130937e+18,18332190,...,0,0,0,0,low,en,1558527745934,NaN,,1130937322109898755
2,2019-05-22 12:23:03,1131173625258815488,@Pelusitaaaa1 Hoi! Enkel KLM/Air France-vluch...,@Pelusitaaaa1 Hoi! Enkel KLM/Air France-vluch...,None,"<a href=""http://www.salesforce.com"" rel=""nofol...",1,1.131171e+18,3.110269e+09,56377143,...,0,0,0,0,low,nl,1558527783108,NaN,,3110268790
3,2019-05-22 12:23:11,1131173660532838400,@wineandthelaw We're concerned to hear this. C...,None,None,"<a href=""http://www.conversocial.com"" rel=""nof...",0,1.131172e+18,1.808266e+08,18332190,...,0,0,0,0,low,en,1558527791518,NaN,,180826577
4,2019-05-22 12:23:12,1131173663745683457,"@wags9000 This is concerning to hear, Ally. Di...",None,None,"<a href=""http://www.conversocial.com"" rel=""nof...",0,1.131129e+18,3.024937e+08,18332190,...,0,0,0,0,low,en,1558527792284,NaN,,302493679
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400978,2020-03-30 18:36:54,1244695110639632386,@misubisumisu We've sent you a DM with a message.,None,None,"<a href=""http://ivoucher.aa.com/snap"" rel=""nof...",0,NaN,1.129592e+18,22536055,...,0,0,0,0,low,en,1585593414350,NaN,,1129591998153990146
400979,2020-03-30 18:38:37,1244695542711693312,@SallyJDavies Hi Sally. Can I just confirm who...,None,None,"<a href=""https://www.hootsuite.com"" rel=""nofol...",0,1.244692e+18,2.384970e+07,20626359,...,0,0,0,0,low,en,1585593517364,NaN,,23849698
400980,2020-03-30 18:39:19,1244695718792699905,@airchive We're thrilled to put in a request a...,None,None,"<a href=""http://ivoucher.aa.com/snap"" rel=""nof...",0,1.244689e+18,2.778661e+09,22536055,...,0,0,0,0,low,en,1585593559345,NaN,'AATeam',2778661370
400981,2020-03-30 18:40:14,1244695952151240705,"@Carinamarie13 Hi there,due to the high volume...","@Carinamarie13 Hi there,due to the high volume...",None,"<a href=""http://www.conversocial.com"" rel=""nof...",1,1.241499e+18,1.427769e+08,18332190,...,0,0,0,0,low,en,1585593614982,NaN,,142776890


In [4]:
def get_tweetarr_to_origin(initial_id):
    
    tweetarr = []

    def get_tweet(nid: int, _tweetarr):
        cur.execute("""SELECT * FROM TWEETS WHERE id == %s""" % nid)
        fetch = cur.fetchall()
        if len(fetch) == 0:
            # ("tweet doesnt exist")
            return {}
        tweet = {tweet_columns[i]:fetch[0][i] for i in range(0, len(tweet_columns))}
        tweet["is_reply"] = (not tweet["in_reply_to_status_id"] is None)
        tweet["is_quote"] = (not tweet["quoted_status_id"] is None)
        _tweetarr.append(tweet)
        return tweet
    
    origin = get_tweet(initial_id, tweetarr)
    i = 0
    while True:
        i += 1
        if origin == {}:
            return {"origin_exists": False}
        if origin["is_reply"]:
            origin = get_tweet(origin["in_reply_to_status_id"], tweetarr)
        elif origin["is_quote"]:
            origin = get_tweet(origin["quoted_status_id"], tweetarr)
        else:
            return {"origin_exists": True, "origin":origin, "arr":tweetarr}
        
conversations_per_airline = {}
for airline in airlines:
    airline_id = airlines[airline]
    cur.execute(prepare_prompt("""SELECT * FROM TWEETS WHERE user_id == %s
    AND (in_reply_to_status_id IS NOT NULL OR quoted_status_id IS NOT NULL)""" % airline_id))        
    airline_tweets = pd.DataFrame(cur.fetchall(), columns=tweet_columns)
    lengths = {}
    s = 0
    print()
    print("getting %s conversations" % airline, end='')
    arr = []
    for row in airline_tweets.iterrows():
        tweet = row[1]
        conversation = get_tweetarr_to_origin(tweet.id)
        if not conversation["origin_exists"]:
            continue
        if not s % 10000:
            print(s, end='')
        s += 1
        arr.append(conversation)
    conversations_per_airline[airline] = arr

        




getting KLM conversations010000
getting AirFrance conversations0
getting British_Airways conversations0100002000030000
getting AmericanAir conversations010000200003000040000
getting Lufthansa conversations0
getting AirBerlin conversations
getting AirBerlin assist conversations
getting easyJet conversations010000
getting RyanAir conversations0
getting SingaporeAir conversations0
getting Qantas conversations0
getting EtihadAirways conversations0
getting VirginAtlantic conversations0

In [5]:
airline_df = pd.DataFrame(conversations_per_airline["KLM"])
airline_df

,origin_exists,origin,arr
0,True,"{'created_at': '2019-05-22 12:58:12', 'id': 11...","[{'created_at': '2019-05-22 13:20:05', 'id': 1..."
1,True,"{'created_at': '2019-05-22 12:58:12', 'id': 11...","[{'created_at': '2019-05-22 13:20:31', 'id': 1..."
2,True,"{'created_at': '2019-05-22 12:58:12', 'id': 11...","[{'created_at': '2019-05-22 13:21:22', 'id': 1..."
3,True,"{'created_at': '2019-05-22 13:35:07', 'id': 11...","[{'created_at': '2019-05-22 13:56:59', 'id': 1..."
4,True,"{'created_at': '2019-05-22 14:56:56', 'id': 11...","[{'created_at': '2019-05-22 14:59:24', 'id': 1..."
...,...,...,...
11948,True,"{'created_at': '2020-03-30 10:08:14', 'id': 12...","[{'created_at': '2020-03-30 12:04:48', 'id': 1..."
11949,True,"{'created_at': '2020-03-30 14:11:34', 'id': 12...","[{'created_at': '2020-03-30 15:32:18', 'id': 1..."
11950,True,"{'created_at': '2020-03-25 18:22:05', 'id': 12...","[{'created_at': '2020-03-30 16:34:18', 'id': 1..."
11951,True,"{'created_at': '2020-03-30 13:54:57', 'id': 12...","[{'created_at': '2020-03-30 17:30:18', 'id': 1..."


In [6]:
from collections import Counter

s = []
for airline in airlines:
    airline_df = pd.DataFrame(conversations_per_airline[airline])
    airline_df["airline"] = airline
    if len(airline_df) < 30:
        continue
    airline_df["length"] = airline_df["arr"].map(len)
    s.append(airline_df)
    
all_df = pd.concat(s)
all_df

,origin_exists,origin,arr,airline,length
0,True,"{'created_at': '2019-05-22 12:58:12', 'id': 11...","[{'created_at': '2019-05-22 13:20:05', 'id': 1...",KLM,2
1,True,"{'created_at': '2019-05-22 12:58:12', 'id': 11...","[{'created_at': '2019-05-22 13:20:31', 'id': 1...",KLM,2
2,True,"{'created_at': '2019-05-22 12:58:12', 'id': 11...","[{'created_at': '2019-05-22 13:21:22', 'id': 1...",KLM,2
3,True,"{'created_at': '2019-05-22 13:35:07', 'id': 11...","[{'created_at': '2019-05-22 13:56:59', 'id': 1...",KLM,2
4,True,"{'created_at': '2019-05-22 14:56:56', 'id': 11...","[{'created_at': '2019-05-22 14:59:24', 'id': 1...",KLM,2
...,...,...,...,...,...
7578,True,"{'created_at': '2020-03-30 12:42:04', 'id': 12...","[{'created_at': '2020-03-30 17:25:46', 'id': 1...",VirginAtlantic,4
7579,True,"{'created_at': '2020-03-30 16:27:26', 'id': 12...","[{'created_at': '2020-03-30 17:33:10', 'id': 1...",VirginAtlantic,2
7580,True,"{'created_at': '2020-03-30 16:36:14', 'id': 12...","[{'created_at': '2020-03-30 17:37:35', 'id': 1...",VirginAtlantic,2
7581,True,"{'created_at': '2020-03-30 17:59:13', 'id': 12...","[{'created_at': '2020-03-30 18:05:01', 'id': 1...",VirginAtlantic,2


In [24]:
counts = all_df.groupby(["airline", "length"]).count()
counts["Count"] = counts["origin"]
counts_noindex = counts.reset_index()
counts

origin_exists  origin   arr  Count
airline   length                                    
AirFrance 2                3260    3260  3260   3260
          3                 111     111   111    111
          4                 182     182   182    182
          5                  17      17    17     17
          6                  18      18    18     18
...                         ...     ...   ...    ...
easyJet   5                 100     100   100    100
          6                  95      95    95     95
          7                  14      14    14     14
          8                   4       4     4      4
          9                   1       1     1      1

[80 rows x 4 columns]

In [39]:
fig = px.histogram(counts_noindex, 
                   x="airline", y="Count", color="length", 
                   title="Histogram of Unique conversations of tweets per airline <br>Ending on airline's last reply/quote")
fig.show()

In [38]:
fig = px.violin(all_df[all_df["length"]>=3], x="airline", 
                title="Violin plot of lengths of unique airline conversations of 3+ tweets ending on airline's last reply/quote",
             y="length", color="airline")
fig.show()